In [64]:
import cv2
import numpy as np

def match_images(train_image, test_image):
    # Initialize ORB detector


    # star = cv2.xfeatures2d.StarDetector_create()
    fast = cv2.FastFeatureDetector_create() 

    # Initiate BRIEF extractor 
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    
    # find the keypoints with STAR

    train_keypoints = fast.detect(train_image, None)
    test_keypoints = fast.detect(test_image, None)
    
    # compute the descriptors with BRIEF
    train_keypoints, train_descriptor = brief.compute(train_image, train_keypoints)
    test_keypoints, test_descriptor = brief.compute(test_image, test_keypoints)   

    # Create a Brute Force Matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    # Match descriptors
    matches = bf.match(train_descriptor, test_descriptor)
    
    matches = sorted(matches, key=lambda x: x.distance)

  # Convert keypoints to point format for RANSAC
    src_pts = np.float32([train_keypoints[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([test_keypoints[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

  # RANSAC for outlier removal
    model, inliers = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

  # Count the inliers (good matches)
    # num_inliers = len(inliers)
    similarity_score = len(inliers)

    return similarity_score


In [65]:
# Load the banana images
train_image = cv2.imread('train_banana.png', cv2.IMREAD_GRAYSCALE)
test_image = cv2.imread('test_banana.png', cv2.IMREAD_GRAYSCALE)

# Match the images
similarity_score = match_images(train_image, test_image)

# Define a threshold for similarity score
threshold = 50

print(similarity_score)
# Determine if the images depict the same fruit or different fruits
if similarity_score >= threshold:
    print("The images depict the same fruit.")
else:
    print("The images depict different fruits.")

325
The images depict the same fruit.


libpng warning: iCCP: known incorrect sRGB profile


In [66]:
fruits = ["apple", "banana", "tomato"]
def classify_fruit(image):
  
  fast = cv2.FastFeatureDetector_create() 
  brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
  # Detect keypoints and compute descriptors for both image and reference images
  test_keypoints = fast.detect(image, None)
  test_keypoints, test_descriptors = brief.compute(image, test_keypoints)

  # Define reference features for each fruit class (manually create or load pre-computed)
  reference_features = {}
  for fruit in fruits:
    # Load reference image for the fruit class (replace with your image loading logic)
    train_image = cv2.imread(f"train_{fruit}.png", cv2.IMREAD_GRAYSCALE)
    # train_image_gray = cv2.cvtColor(train_image, cv2.COLOR_BGR2GRAY)

    train_keypoints = fast.detect(image, None)
    train_keypoints, train_descriptors = brief.compute(train_image, train_keypoints)

    # reference_kp, reference_des = orb.detectAndCompute(reference_gray, None)
    reference_features[fruit] = train_descriptors

  # Matching with Brute-Force Matcher and Hamming distance
  bf = cv2.BFMatcher(cv2.NORM_HAMMING)
  best_matches = []
  best_fruit = None

  for fruit, train_descriptors in reference_features.items():
    matches = bf.knnMatch(test_descriptors, train_descriptors, k=2)
    good_matches = [m for m,n in matches if m.distance < 0.7*n.distance]  # Adjust ratio based on descriptor

    # Consider number of good matches for classification
    if len(good_matches) > len(best_matches):
      best_matches = good_matches
      best_fruit = fruit

  # Confidence threshold (optional)
  confidence_threshold = 0.5 * len(test_keypoints)  # Adjust threshold based on image complexity
  if len(best_matches) / len(test_keypoints) > confidence_threshold:
    return best_fruit
  else:
    return "Unknown Fruit"

In [67]:
fruit_label = classify_fruit(test_image)
print(fruit_label)

error: OpenCV(4.9.0) /io/opencv/modules/core/src/batch_distance.cpp:274: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'batchDistance'
